In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json 
import tensorflow_text as tf_text


In [34]:
with open("E:/MLCourse/foxBot/foxPython/word_2_vec/data/new_vocabs_weights.json","r",encoding="utf-8") as f:
    new_vocab_weights=json.load(f)

with open("E:/MLCourse/foxBot/foxPython/word_2_vec/data/vocabs_weights.json","r",encoding="utf-8") as f:
    vocab_weights=json.load(f)

textlist=[]
nerTag=[]
with open("E:/MLCourse/foxBot/foxPython/EntitiesTagger/tagFolder/tagtrain.json","r",encoding="utf-8") as f:
    data=json.load(f)
for i in data:
    textlist.append(i["sentence"])
    nerTag.append(i["tag"])
data

[{'sentence': 'Tôi đang làm Toán', 'tag': ['o', 'o', 'o', 'B-mh']},
 {'sentence': 'Văn học đôi khi dễ buồn ngủ',
  'tag': ['B-mh', 'I-mh', 'o', 'o', 'o', 'o', 'o']},
 {'sentence': 'Tố Hữu là một trong tác giả nổi tiếng trong văn học',
  'tag': ['B-name',
   'I-name',
   'o',
   'o',
   'o',
   'o',
   'o',
   'o',
   'o',
   'o',
   'o',
   'o']},
 {'sentence': 'Toán lớp 12 thực sự rất hay',
  'tag': ['B-mh', 'o', 'o', 'o', 'o', 'o', 'o']},
 {'sentence': 'Tôi tên là Kiên', 'tag': ['o', 'o', 'o', 'B-name']},
 {'sentence': 'Tên tôi là John', 'tag': ['o', 'o', 'o', 'B-name']},
 {'sentence': 'Vật lý là nổi ác mộng',
  'tag': ['B-mh', 'I-mh', 'o', 'o', 'o', 'o']},
 {'sentence': 'Newton khám phá ra lực hấp dẫn',
  'tag': ['B-name', 'o', 'o', 'o', 'B-obj', 'I-obj', 'I-obj']},
 {'sentence': 'Pomodoro giúp tiết kiệm thời gian của người học hơn',
  'tag': ['B-obj', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']},
 {'sentence': 'Feynman là nhà vật lý lừng danh nhất thời đại',
  'tag': ['B-name', 'o

In [35]:
TextTokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=20000,lower=True,oov_token="<UNK>")
TextTokenizer.fit_on_texts(textlist)
tokenizedText=TextTokenizer.texts_to_sequences(textlist)

TagTokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=20000,lower=False,oov_token="<UNK>",filters="",)
TagTokenizer.fit_on_texts(nerTag)
tokenizedTag=TagTokenizer.texts_to_sequences(nerTag)

textFinal=tf.keras.preprocessing.sequence.pad_sequences(tokenizedText)
tagFinal=tf.keras.preprocessing.sequence.pad_sequences(tokenizedTag)



In [37]:
num_tokens = len(TextTokenizer.word_index) + 1
embedding_dim = 128
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in TextTokenizer.word_index.items():
    embedding_vector = new_vocab_weights.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

embedding_layer = tf.keras.layers.Embedding(
    num_tokens,
    128,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)



Converted 205 words (35 misses)


In [38]:
def LSTMModel():
    inputs=tf.keras.layers.Input(shape=(textFinal.shape[1],),dtype="int32")
    Embedding=embedding_layer(inputs)
    # Embedding=tf.keras.layers.Embedding(len(TextTokenizer.word_index)+1,5)(inputs)
    x=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True))(Embedding)
    x=tf.keras.layers.Dense(12,activation="relu")(x)
    # classifer=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(TagTokenizer.word_index)+1,activation="softmax"))(x)
    classifer=tf.keras.layers.Dense(len(TagTokenizer.word_index)+1,activation="softmax")(x)
    return tf.keras.Model(inputs,classifer)

In [39]:
myModel=LSTMModel()
myModel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.025),loss="sparse_categorical_crossentropy",metrics=["sparse_categorical_accuracy"])

In [41]:
myModel.fit(textFinal,tagFinal,batch_size=8,epochs=250,validation_split=0.2)

Epoch 1/250
5/5 [==============================] - 4s 218ms/step - loss: 2.0250 - sparse_categorical_accuracy: 0.5688 - val_loss: 0.7336 - val_sparse_categorical_accuracy: 0.8591
Epoch 2/250
5/5 [==============================] - 0s 19ms/step - loss: 0.7400 - sparse_categorical_accuracy: 0.8452 - val_loss: 0.5203 - val_sparse_categorical_accuracy: 0.8955
Epoch 3/250
5/5 [==============================] - 0s 18ms/step - loss: 0.5544 - sparse_categorical_accuracy: 0.8698 - val_loss: 0.5077 - val_sparse_categorical_accuracy: 0.9000
Epoch 4/250
5/5 [==============================] - 0s 17ms/step - loss: 0.4662 - sparse_categorical_accuracy: 0.8784 - val_loss: 0.4669 - val_sparse_categorical_accuracy: 0.9091
Epoch 5/250
5/5 [==============================] - 0s 17ms/step - loss: 0.3722 - sparse_categorical_accuracy: 0.8845 - val_loss: 0.4894 - val_sparse_categorical_accuracy: 0.9091
Epoch 6/250
5/5 [==============================] - 0s 18ms/step - loss: 0.3085 - sparse_categorical_accuracy:

In [67]:
myModel.save("E:/MLCourse/foxBot/foxPython/EntitiesTagger/TagModel")

INFO:tensorflow:Assets written to: E:/MLCourse/TF2/Notebook/SmartFennec/NER file/TagModel\assets


INFO:tensorflow:Assets written to: E:/MLCourse/TF2/Notebook/SmartFennec/NER file/TagModel\assets


In [43]:
def insert_text(text):
    text_tokenized=TextTokenizer.texts_to_sequences([text])
    finaltext=tf.keras.preprocessing.sequence.pad_sequences(text_tokenized,maxlen=textFinal.shape[1])
    return finaltext


In [68]:
rs=np.round(myModel.predict(insert_text("Tôi tên là Kiên")),decimals=2)
txt=[]
for i in rs[0]:
    txt.append(tagdict[np.argmax(i)])
    # use argmax 
print(txt)
rs

['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'o', 'o', 'o', 'B-name']


array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
